In [1]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import gridspec as mpgs
import matplotlib as mpl
import numpy as np
import scipy as sp
from skimage import transform as skt
import os
import time
import multiprocessing as mp
import skimage.filters as skf
from scipy import ndimage as scnd
import hdf5storage as h5
from matplotlib_scalebar.scalebar import ScaleBar,SI_LENGTH_RECIPROCAL
import math
import numba
import sys
import pyfftw
import pymcr
%matplotlib inline
plt.rcParams['image.cmap'] = 'magma'
plt.rcParams.update({'font.size': 25})
plt.rcParams.update({'font.weight': 'bold'})
plt.rcParams.update({'figure.autolayout': True})

In [2]:
import sys
sys.path.append('/home/cades/Codes/stemtools')
import stemtools as st
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from matplotlib.colors import LinearSegmentedColormap as LSC
lin_burd = LSC.from_list('list', [(0, 35/255, 156/255), (0.95,0.95, 0.95),(249/255,56/255,34/255)], N=256)

In [5]:
st.tic()
datRhPt = h5.loadmat('/home/cades/Data4D_2/Debangshu/Core@Shell NBED/Nion Swift Library 20190810 - EH-83-I Rh@Pt Thick/RhPtThick.mat')
st.toc()

Elapsed time: 26.222679 seconds.



In [ ]:
json1 = '/home/cades/Data4D_2/Debangshu/Core@Shell NBED/Nion Swift Library 20190810 - EH-83-I Rh@Pt Thick/Particle1_ACSTEM_00.json'
json2 = '/home/cades/Data4D_2/Debangshu/Core@Shell NBED/Nion Swift Library 20190810 - EH-83-I Rh@Pt Thick/Particle2_ACSTEM_00.json'
with open(json1) as json_1:
    info_1 = json.load(json_1)
with open(json2) as json_2:
    info_2 = json.load(json_2)

In [ ]:
calib1 = info_1['spatial_calibrations'][0]['scale']
calib1_units = info_1['spatial_calibrations'][0]['units']

In [ ]:
calib1

In [ ]:
datRhPt.keys()

In [ ]:
image00 = datRhPt['particle1_00']
image90 = datRhPt['particle1_90']
imageDC = datRhPt['particle1_im']

In [ ]:
calib1_units

In [ ]:
font = {'weight' : 'bold',
        'size'   : 50}

mpl.rc('font', **font)
plt.figure(figsize=(20,30))

plt.subplot(3,2,1)
plt.imshow(scnd.rotate(image00,90))
scalebar = ScaleBar(calib1,calib1_units)
scalebar.location = 'lower right'
scalebar.box_alpha = 0
scalebar.color = 'w'
plt.gca().add_artist(scalebar)
plt.axis('off')

plt.subplot(3,2,2)
plt.imshow(scnd.gaussian_filter(scnd.rotate(image00,90),2))
scalebar = ScaleBar(calib1,calib1_units)
scalebar.location = 'lower right'
scalebar.box_alpha = 0
scalebar.color = 'w'
plt.gca().add_artist(scalebar)
plt.axis('off')

plt.subplot(3,2,3)
plt.imshow(image90)
scalebar = ScaleBar(calib1,calib1_units)
scalebar.location = 'lower right'
scalebar.box_alpha = 0
scalebar.color = 'w'
plt.gca().add_artist(scalebar)
plt.axis('off')

plt.subplot(3,2,4)
plt.imshow(scnd.gaussian_filter(image90,2))
scalebar = ScaleBar(calib1,calib1_units)
scalebar.location = 'lower right'
scalebar.box_alpha = 0
scalebar.color = 'w'
plt.gca().add_artist(scalebar)
plt.axis('off')

plt.subplot(3,2,5)
plt.imshow(scnd.rotate(imageDC,90))
scalebar = ScaleBar(calib1,calib1_units)
scalebar.location = 'lower right'
scalebar.box_alpha = 0
scalebar.color = 'w'
plt.gca().add_artist(scalebar)
plt.axis('off')

plt.subplot(3,2,6)
plt.imshow(scnd.gaussian_filter(scnd.rotate(imageDC,90),2))
scalebar = ScaleBar(calib1,calib1_units)
scalebar.location = 'lower right'
scalebar.box_alpha = 0
scalebar.color = 'w'
plt.gca().add_artist(scalebar)
plt.axis('off')

plt.tight_layout()

plt.savefig('Results_ADF/Particle1_Raw_Images.pdf')

To ensure only the nanoparticle is picked, we will do a digital selected area electron diffraction - we will pick two diffraction spots and then mask the image so that we will analyze only the image where the diffraction spots came from.

In [ ]:
image00 = scnd.gaussian_filter(scnd.rotate(image00,90),2)
image90 = scnd.gaussian_filter(image90,2)
imageDC = scnd.rotate(imageDC,90)

In [ ]:
imageDC.shape

In [ ]:
circ_c=(985,985)
circ_y=(985,905)
circ_x=(1065,985)
st.afit.find_diffraction_spots(image=imageDC,
                               circ_c=circ_c,
                               circ_y=circ_y,
                               circ_x=circ_x)

In [ ]:
def define_reference(image,
                     A,B,C,D):
    xx,yy = np.meshgrid(np.arange(imageDC.shape[1]),np.arange(imageDC.shape[0]))
    m_AB = (A[1] - B[1])/(A[0] - B[0])
    c_AB = A[1] - (m_AB*A[0])
    m_BC = (B[1] - C[1])/(B[0] - C[0])
    c_BC = B[1] - (m_BC*B[0])
    m_CD = (C[1] - D[1])/(C[0] - D[0])
    c_CD = C[1] - (m_CD*C[0])
    m_DA = (D[1] - A[1])/(D[0] - A[0])
    c_DA = D[1] - (m_DA*D[0])
    ref_reg = np.logical_and(np.logical_and((yy > (m_AB*xx) + c_AB),((yy - c_BC)/m_BC > xx)),
                             np.logical_and((yy < (m_CD*xx) + c_CD),((yy - c_DA)/m_DA < xx)))
    plt.figure(figsize=(15,15))
    plt.imshow(st.util.image_normalizer(image)+0.33*ref_reg)
    plt.annotate(A, (A[0]/image.shape[0], (1 - A[1]/imageDC.shape[1])), textcoords='axes fraction', size=15,c='w')
    plt.annotate(B, (B[0]/image.shape[0], (1 - B[1]/imageDC.shape[1])), textcoords='axes fraction', size=15,c='w')
    plt.annotate(C, (C[0]/image.shape[0], (1 - C[1]/imageDC.shape[1])), textcoords='axes fraction', size=15,c='w')
    plt.annotate(D, (D[0]/image.shape[0], (1 - D[1]/imageDC.shape[1])), textcoords='axes fraction', size=15,c='w')
    plt.scatter(A[0],A[1])
    plt.scatter(B[0],B[1])
    plt.scatter(C[0],C[1])
    plt.scatter(D[0],D[1])
    plt.axis('off')
    return ref_reg

In [ ]:
A = (920,750)
B = (1300,740)
C = (1290,1080)
D = (900,1100)
ref_matrix = define_reference(imageDC,A,B,C,D)

In [ ]:
def circ_to_G(circ_pos,image):
    g_vec = np.zeros(2)
    g_vec[0] = ((circ_pos[1] - (0.5*image.shape[0]))/image.shape[0])
    g_vec[1] = ((circ_pos[0] - (0.5*image.shape[1]))/image.shape[1])
    return g_vec

def G_to_circ(g_vec,image):
    circ_pos = np.zeros(2)
    circ_pos[1] = (g_vec[0]*image.shape[0]) + (0.5*image.shape[0])
    circ_pos[0] = (g_vec[1]*image.shape[1]) + (0.5*image.shape[1])
    return circ_pos

def g_matrix(g_vector,image):
    x_fourier = np.fft.fftshift(np.fft.fftfreq(image.shape[1]))
    y_fourier = np.fft.fftshift(np.fft.fftfreq(image.shape[0]))
    P_vec,Q_vec = np.meshgrid(x_fourier,y_fourier)
    G_Matrix = Q_vec*g_vector[0] + P_vec*g_vector[1]
    return (2*np.pi)*G_Matrix

def p_matrix(g_vector,image):
    circ_pos = G_to_circ(g_vector,image)
    circ_rad = np.amin(0.01*np.asarray(image.shape))
    circ_mask = st.util.make_circle(imageDC.shape,circ_pos[0],circ_pos[1],circ_rad)
    G_matrix = np.angle(np.fft.ifft2(circ_mask*np.fft.fftshift(np.fft.fft2(image))))
    P_matrix = G_matrix - g_matrix(g_vector,image)
    return P_matrix

In [ ]:
gvec_y = circ_to_G(circ_y,imageDC)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(ref_matrix*(p_matrix(circ_to_G(circ_x,imageDC),imageDC)))

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(ref_matrix[:,0:-1]*(np.diff(p_matrix(circ_to_G(circ_x,imageDC),imageDC))))

In [ ]:
whos